
---

## 🧩 **Problem Statement: Translate a PDF Story into Hindi Using LangChain**

---

### 📌 Task:

You are given a PDF file containing an English story. Your goal is to:

1. **Upload and read the PDF file** using `PyMuPDFLoader` from LangChain.
- PDF File link - [Click here](https://drive.google.com/file/d/1hNkcxV4-T5I-wdejnnmKKjU1oinLdGnw/view?usp=sharing)
Download PDF and store in local.
2. **Extract text** from all pages of the PDF.
3. Use **RecursiveCharacterTextSplitter** to divide the text into **chunks of 30 characters** (no overlap).
4. For each chunk, use an **LLM** (like OpenAI’s ChatGPT via `ChatOpenAI`) to **translate the chunk into Hindi**.
5. **Display the original English chunk and its Hindi translation side by side.**

---

### 🧪 Input:

* A PDF file (e.g., `sample_story.pdf`) containing a short English story.

---

### ✅ Output:

* Print a list of **translated Hindi chunks**, one per line.
* Each line should include the **original English chunk** and the **translated Hindi output**.

Example:

```
Chunk 1 (EN): Once upon a time in a small vi  
Translation (HI): एक समय की बात है, एक छोटे से गा

Chunk 2 (EN): llage nestled between the hill  
Translation (HI): गांव जो पहाड़ियों के बीच बसा
```

---

### 🔧 Tools You Must Use:

* `langchain.document_loaders.PyMuPDFLoader` for reading the PDF
* `langchain.text_splitter.RecursiveCharacterTextSplitter`
* `langchain.chat_models.ChatOpenAI` (or any supported LLM)

---

### 📎 Sample File:

You can use this file for testing:
📄 [Download `sample_story.pdf`](sandbox:/mnt/data/sample_story.pdf)

---



In [ ]:
!pip install -qU langchain langchain-google-genai langchain-community openai pymupdf tqdm

In [ ]:
import os
from getpass import getpass


if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google Gemini API key: ")

In [ ]:
from google.colab import files

uploaded = files.upload()  # pick your file (e.g., sample_story.pdf)
pdf_path = list(uploaded.keys())[0]
print("Using:", pdf_path)


Saving Sep2025Resume.pdf to Sep2025Resume.pdf
Using: Sep2025Resume.pdf


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1) Load the PDF
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()  # list[Document], one per page by default

# 2) Extract full text and split into 30-char chunks (no overlap)
full_text = "\n".join(d.page_content for d in docs)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=0,      # <- no overlap
    separators=["\n\n", "\n", " "]  # natural breakpoints first, then fallback
)
chunks = [c for c in splitter.split_text(full_text) if c.strip()]

print(f"Total chunks: {len(chunks)}")

# import { RecursiveCharacterTextSplitter } from "@langchain/textsplitters";

# const splitter = new RecursiveCharacterTextSplitter({ chunkSize: 100, chunkOverlap: 0 })
# const texts = splitter.splitText(document)

Total chunks: 173


In [ ]:
# 3) Init LLM (assignment asks ChatOpenAI). Temperature=0 for deterministic-ish output
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)  # Use any ChatOpenAI-supported model
# If you strictly need "ChatGPT via ChatOpenAI" and your account supports it,
# you can use model="gpt-4o" / "gpt-4o-mini" / "gpt-4.1-mini" etc.

# 4) Translate chunk-by-chunk
translated = []
system_prompt = (
    "You are a precise translator. Translate the given English text to Hindi. "
    "Preserve meaning and tone. Do not add or omit content. Return only the translation."
)

for chunk in tqdm(chunks, desc="Translating"):
    user_prompt = f"Translate to Hindi:\n\n{chunk}"
    resp = llm.invoke([{"role": "system", "content": system_prompt},
                       {"role": "user", "content": user_prompt}])
    hi = resp.content.strip()
    translated.append((chunk, hi))

# 5) Print: English + Hindi side by side (as requested)
print("\n===== English → Hindi (one per line) =====\n")
for i, (en, hi) in enumerate(translated, start=1):
    print(f"Chunk {i} (EN): {en}")
    print(f"Translation (HI): {hi}\n")

Translating:  52%|█████▏    | 11/21 [00:32<00:20,  2.02s/it]WARNING:langchain_google_genai.chat_models:Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 31.808346166s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more abou


===== English → Hindi (one per line) =====

Chunk 1 (EN): Once upon a time in a small village nestled
Translation (HI): एक समय की बात है, एक छोटे से गाँव में जो बसा हुआ था

Chunk 2 (EN): between the hills, there lived a curious little
Translation (HI): पहाड़ियों के बीच, एक जिज्ञासु छोटा रहता था

Chunk 3 (EN): girl named
Translation (HI): नाम की लड़की

Chunk 4 (EN): Meera. She loved asking questions and exploring
Translation (HI): मीरा। उसे सवाल पूछना और खोजबीन करना बहुत पसंद था।

Chunk 5 (EN): the woods near her home.
Translation (HI): उसके घर के पास का जंगल।

Chunk 6 (EN): Every morning, Meera would set out on a new
Translation (HI): हर सुबह, मीरा एक नई पर निकल पड़ती थी।

Chunk 7 (EN): adventure, talking to birds and watching
Translation (HI): रोमांच, पक्षियों से बातें करना और देखना

Chunk 8 (EN): squirrels
Translation (HI): गिलहरियाँ

Chunk 9 (EN): dance on tree branches. Her favorite story was
Translation (HI): पेड़ की डालियों पर नाचना। उसकी पसंदीदा कहानी थी

Chunk 10 (EN): about a